In [9]:
import csv
import cv2
import numpy as np
import random
import os
import torchvision
import torchvision.transforms as transforms
from torchvision import io
from tqdm import tqdm
import torchvision.models as models
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [10]:
alphabets = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
alphabets2index = {alphabet:i for i, alphabet in enumerate(alphabets)}
print(alphabets2index)
index2alphabets = {i: alphabet for i, alphabet in enumerate(alphabets)}

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '0': 52, '1': 53, '2': 54, '3': 55, '4': 56, '5': 57, '6': 58, '7': 59, '8': 60, '9': 61}


In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/captcha-hacker-2023-spring.zip
/kaggle/dataset/sample_submission.csv
/kaggle/dataset/test/task2/dreFISY5A8elpMKK.png
/kaggle/dataset/test/task2/13Wfs6YbjJTVlClY.png
/kaggle/dataset/test/task2/IN9vKdhs2RhWsDiw.png
...
/kaggle/dataset/test/task3/kLjULZiHIj8uWdHR.png
/kaggle/dataset/test/task3/fWkcA63LEHbSrJIN.png
/kaggle/dataset/test/task3/DBO0TS2J09rA8fdL.png
...
/kaggle/dataset/test/task1/gXb4I8nnMD8wp8Is.png
/kaggle/dataset/test/task1/mr6b36cQdcoidaGh.png
/kaggle/dataset/test/task1/pTKVsDqscDb50LQV.png
...
/kaggle/dataset/train/annotations.csv
/kaggle/dataset/train/task2/xKdEhdwRMuTpYJN9.png
/kaggle/dataset/train/task2/64tSL8zTFH8AN7L5.png
/kaggle/dataset/train/task2/mc2JOVUgrIr7RQGo.png
...
/kaggle/dataset/train/task3/7MWp2K915drw2FLZ.png
/kaggle/dataset/train/task3/RwV12vmr0J9xc3e9.png
/kaggle/dataset/train/task3/G8focQMqdQ7MqP7Q.png
...
/kaggle/dataset/train/task1/t92un1ppRzA4u6bx.png
/kaggle/dataset/train/task1/VIBOQUyPFDroWDwl.png
/kaggle/dataset/train/task1/plVlZnknj2pxg

In [12]:
TRAIN_PATH = "/kaggle/dataset/train"
TEST_PATH = "/kaggle/dataset/test"
device = 'cuda'

# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster if your using kaggle's enviroment

In [13]:
transform_1= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # 添加灰階轉換
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # 只有一個通道的均值和標準差
])


In [14]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        # img = cv2.resize(img, (32, 32))
        # img = np.mean(img, axis=2)
        img = transform_1(img)
        if self.return_filename:
            return torch.FloatTensor(img ), filename
        else:
            return torch.FloatTensor(img ), alphabets2index[label]

    def __len__(self):
        return len(self.data)

In [15]:
class ResNet18_1(nn.Module):
    def __init__(self):
        super(ResNet18_1, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 62)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [16]:
test_data = []
model = ResNet18_1().to(device)
PATH_1 = "/content/drive/MyDrive/final_PR_weight/model_1.pt"
with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

# !rm -f submission.csv
if os.path.exists('submission.csv'):
    file = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(file)
else:
    file = open('submission.csv', 'w', newline='')
    csv_writer = csv.writer(file)
    csv_writer.writerow(["filename", "label"])

test_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

model.load_state_dict(torch.load(PATH_1))

print( len(test_data) )

model.eval()
cnt = 0
for image, filenames in test_dl:
    image = image.to(device)
    
    pred = model(image)
    pred = torch.argmax(pred, dim=1)
    
    for i in range(len(filenames)):
      csv_writer.writerow([filenames[i], alphabets[pred[i].item()]])
      cnt += 1
print(cnt) # 6500



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the wo

10001
6500


In [17]:
transform_2= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # 添加灰階轉換
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # 只有一個通道的均值和標準差
])

In [18]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = transform_2(img)
        if self.return_filename:
            return torch.FloatTensor( img ), filename
        else:
            char_label = np.zeros(124)
            j = 0
            for i in label :
               char_label[alphabets2index[i]+j] = 1
               j = j + 62
            return torch.FloatTensor(img ), char_label

    def __len__(self):
        return len(self.data)

In [19]:
class ResNet18_2(nn.Module):
    def __init__(self):
        super(ResNet18_2, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 124)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [20]:
# if os.path.exists('submission.csv'):
#     csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
# else:
#     csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
#     csv_writer.writerow(["filename", "label"])

test_ds = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

model = ResNet18_2().to(device)
PATH_2 = "/content/drive/MyDrive/final_PR_weight/model_2.pt"
model.load_state_dict(torch.load(PATH_2))
model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model(image)
    pred = pred.data.cpu().numpy()
    pred_chars = list()
    for j in range(pred.shape[0]):
      ans = ""
      for k in range(2) :
        max = -10000
        index = 0
        for i in range(62):
          if pred[j][i+k*62] > max:
            index=i
            max=pred[j][i+k*62]
        ans += alphabets[index]
      pred_chars.append(ans)
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], pred_chars[i]])
        cnt += 1
print(cnt) # 6500


9000


In [21]:
transform_3= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),  # 添加灰階轉換
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # 只有一個通道的均值和標準差
])

In [22]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        # img = cv2.resize(img, (32, 32))
        # img = np.mean(img, axis=2)
        img = transform_3(img)
        if self.return_filename:
            return torch.FloatTensor(img) , filename
        else:
            char_label = np.zeros(248)
            j = 0
            for i in label :
               char_label[alphabets2index[i]+j] = 1
               j = j + 62
            return torch.FloatTensor(img), char_label

    def __len__(self):
        return len(self.data)

In [23]:
class ResNet18_3(nn.Module):
    def __init__(self):
        super(ResNet18_3, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 248)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [24]:
# if os.path.exists('submission.csv'):
#     csv_writer = csv.writer(open('submission.csv', 'a', newline=''))
# else:
#     csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
#     csv_writer.writerow(["filename", "label"])
test_ds = Task3Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)
model = ResNet18_3().to(device)
PATH_3 = "/content/drive/MyDrive/final_PR_weight/model_3.pt"
model.load_state_dict(torch.load(PATH_3))

model.eval()
for image, filenames in test_dl:
    image = image.to(device)
    pred = model(image)
    pred = pred.data.cpu().numpy()
    pred_chars = list()
    for j in range(pred.shape[0]):
      ans = ""
      for k in range(4) :
        max = -10000
        index = 0
        for i in range(62):
          if pred[j][i+k*62] > max:
            index=i
            max=pred[j][i+k*62]
        ans += alphabets[index]
      pred_chars.append(ans)
    for i in range(len(filenames)):
      csv_writer.writerow([filenames[i], pred_chars[i]])
      cnt += 1


print(cnt) # 6500


file.close()

10000
